In [ ]:
# !pip install --upgrade pip

In [ ]:
# !pip install -r requirements.txt

In [ ]:
#!pip install boto3 --upgrade

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

c:\Users\yewo2\.conda\envs\kobert_\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yewo2\.conda\envs\kobert_\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [54]:
# https://jimmy-ai.tistory.com/164

## CPU
device = torch.device("cpu")

ko_model = torch.load('model.pt', map_location=device) # input으로 저장된 디렉토리만 지정하면 완료

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [6]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. d:\GitHub\KoBERT\.cache\kobert_v1.zip
using cached model. d:\GitHub\KoBERT\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. d:\GitHub\KoBERT\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
max_len = 100
batch_size = 64

In [56]:
# https://tech-diary.tistory.com/31

import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  res_list = [] # 결과 값을 담을 리스트
  for c in comment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['text', 'label']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=0)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = ko_model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 일반, 광고는 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정(광고)
        value = 1
      res_list.append(value)

  return res_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

In [57]:
test_ = ['여러 클렌징도구기계 써봤는데 요거 느무 좋다   아 잇치치약도 굿       프라엘초음파클렌저 프라엘 클렌징기계 써모슈티컬 잇치', 
         '이히히히힣힠        대놓고 자랑중입니다    프라엘 남편최고', 
         'lgpral 4th EVENT 코어탄력을 세우는 LG프라엘 공식판매처 알로이비즈 입니다 5월 스페셜 이벤트 이 게시글에 댓글과 함께 친구를 태그 해주세요 추첨을 통해 프라엘 듀얼모션클렌저 레꼴뜨 스마일베이커미니 메디힐 마스크팩을 선물로 드립니다 참여방법 lgpral 팔로우 꾸욱 이벤트 게시글 좋아요 누르고 친구 태그와 함께 댓글 남기기 리그램 하신 후에 엘지프라엘 알로이비즈 프라엘 lg프라엘 이벤트 인스타이벤트 선물이벤트 선물증정 등 해시태그 달기 Tip 지인 태그 여러명 하시면 당첨확률 UP 증정선물 프라엘 듀얼모션클렌저 1명 색상랜덤 레꼴뜨 스마일베이커미니 2명 색상랜덤 메디힐 마스크팩 10매 8명 종류랜덤 이벤트 기간 517527 당첨자 발표 67 금 당첨자 수 11명 선물 증정은 당첨자 발표 후 다음날 발송해 드립니다 궁금하신 점은 언제든지 DM으로 문의 주세요 2019년엔프라엘로더욱아름다워지세요 LG프라엘 풀세트하나면 내일더예쁨예약 프라엘 엘지프라엘 lg프라엘 led마스크 프라엘마스크 피부관리 llg프라엘마스크 엘지프라엘더마led마스크 이나영마스크 이나영 뷰티디바이스 이벤트 리그램이벤트 리그램 선물이벤트 경품이벤트']

In [73]:
test_ = ['노트북 뭐 살까 고민하는 친구들~~! 은하수 이온 추천해! 발열도 적고 15인치에 선명하다. #협찬아님']

In [74]:
result_ = getSentimentValue(test_)
result_

[1]

## model summary

In [ ]:
# https://velog.io/@springkim/pytorch-model-summary

In [ ]:
# !pip install pytorch-model-summary

In [13]:
import torch
import torchvision
import pytorch_model_summary

In [14]:
net = torchvision.models.alexnet()

# show_input : True -> 입력크기, False -> 출력크기
print(pytorch_model_summary.summary(net, torch.zeros(1, 3, 256, 256), show_input=True))
print(pytorch_model_summary.summary(net, torch.zeros(1, 3, 256, 256), show_input=False))

-----------------------------------------------------------------------------
           Layer (type)          Input Shape         Param #     Tr. Param #
               Conv2d-1     [1, 3, 256, 256]          23,296          23,296
                 ReLU-2      [1, 64, 63, 63]               0               0
            MaxPool2d-3      [1, 64, 63, 63]               0               0
               Conv2d-4      [1, 64, 31, 31]         307,392         307,392
                 ReLU-5     [1, 192, 31, 31]               0               0
            MaxPool2d-6     [1, 192, 31, 31]               0               0
               Conv2d-7     [1, 192, 15, 15]         663,936         663,936
                 ReLU-8     [1, 384, 15, 15]               0               0
               Conv2d-9     [1, 384, 15, 15]         884,992         884,992
                ReLU-10     [1, 256, 15, 15]               0               0
              Conv2d-11     [1, 256, 15, 15]         590,080         590,08